<a href="https://colab.research.google.com/github/jhaRahul2102/Book-recommendation-using-collaborative-approach/blob/main/Book_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing modules**

In [7]:
import numpy as np
import pandas as pd

#-----------------------------------------------------------------------------------------

**Loading dataset**

In [8]:
#-------------------------------------------- Books -----------------------------------------------

books=pd.read_csv('/content/drive/MyDrive/Books.csv',on_bad_lines='skip',encoding='latin-1')

#------------------------------------------ User information --------------------------------------------
users=pd.read_csv('/content/drive/MyDrive/Users.csv',on_bad_lines='skip',encoding='mac_latin2')

#-------------------------------------------- Rating ---------------------------------------------
ratings=pd.read_csv('/content/drive/MyDrive/Ratings.csv',on_bad_lines='skip',encoding='mac_latin2')

#--------------------------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Data insight

In [9]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


**Columns of books dataset**

In [10]:
print('Column in books dataset:-',end=' ')
books.columns

Column in books dataset:- 

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

**Dropping unnecessary columns in books dataset:-**
* Columns like img url is not needed for recommendation:-

-------------------------------------------------------------------------------------

**Shape of books columns:-**

In [11]:
print('Shape of books columns:-',end=' ')
books.shape

Shape of books columns:- 

(271360, 8)

------------------------------------------------------------------------------------------
**Statistical description of books dataset:-**

In [12]:
books.describe()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
count,271360,271360,271359,271360,271358,271360,271360,271357
unique,271360,242135,102023,202,16807,271044,271044,271041
top,0195153448,Selected Poems,Agatha Christie,2002,Harlequin,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/225307649X.0...
freq,1,27,632,13903,7535,2,2,2


-------------------------------------------------------------------------------------------------------

In [13]:
print('Shape of users dataset',end=' ')
users.shape

Shape of users dataset 

(278858, 3)

----------------------------------------------------------------------------------------------

In [14]:
print('Shape of ratings dataset',end=' ')
ratings.shape

Shape of ratings dataset 

(1149780, 3)

--------------------------------------------------------------------------------------

In [15]:
users.head(3)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN


---------------------------------------------------------------------------------------

In [16]:
ratings.head(3)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0


 # **Preprocessing of datasets:-**

-----------------------------------------------------------------------------------

In [17]:
# Dropping unnecessary columns:-

books.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'],axis=1,inplace=True)

---------------------------------------------------------------------------------------

**No of user that reviewed the books:-**

We are only including user who have review more than 50.

In [18]:
y=ratings['User-ID'].value_counts()>200
index=y[y].index
ratings=ratings[ratings['User-ID'].isin(index)]

------------------------------------------------------------------------------------------

**Merging rating and book columns:-**

In [19]:
book_r=ratings.merge(books,on='ISBN')

------------------------------------------------------------------------------------------------------------

**Filtering review to be numbe rof review more than 50 review per book.**

In [20]:
book_r_b=book_r.groupby(['Book-Title'])['Book-Rating'].count().reset_index()
book_r=book_r.merge(book_r_b,on='Book-Title').rename(columns={'Book-Rating_y':'Total-no-of-Rating'})


In [21]:
book_r_b=book_r[book_r['Total-no-of-Rating']>50]

---------------------------------------------------------------------------------------------

**Dropping duplicate**

In [22]:
book_r_b=book_r_b.drop_duplicates(['Book-Title','User-ID'])

In [23]:
book_r_b.shape

(57952, 8)

--------------------------------------------------------------------------------------------

**Renaming column name**

In [24]:
book_r_b=book_r_b.rename(columns={'Book-Rating_x':'Rating'})

In [25]:
book_r_b.dropna(inplace=True)

-------------------------------------------------------------------------------------------------------

# Creating pivot table between user and book title.

In [26]:
final=book_r_b.pivot_table(values='Rating',columns='User-ID',index='Book-Title')
final.fillna(0,inplace=True)

---------------------------------------------------------------------------------------------------

**Creating sparse matrix to reduce the matrix complexity**

In [27]:
from scipy.sparse import csr_matrix
book_sparse=csr_matrix(final)

-------------------------------------------------------------------------------------------------------------------------------

**Model processing**

In [28]:
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(algorithm='brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [33]:
def recommend_book(book):
  index=np.where(final.index==book)[0][0]
  distances,suggestion=model.kneighbors(final.iloc[index,:].values.reshape(1,-1),n_neighbors=6)
  for i in range(len(suggestion)):
    print(final.index[suggestion[i]][1:])

#- Book recommending -----------

In [34]:
recommend_book('The Chamber')

Index(['The Cradle Will Fall', 'Exclusive', 'No Safe Place', 'Pleading Guilty',
       'Secrets'],
      dtype='object', name='Book-Title')
